# Contexto ✒️

Esse notebook se refere a um estudo sobre a criação de Agents, que podem ser compreendidos como inteligências artificiais com capacidade de ação e interação com o meio, retirando ou enviando para ele informação.

Tais inteligências artificiais alteram o status quo antes vigente de seu uso passivo no qual, por exemplo, seus outputs só eram gerados mediante a presença de uma pessoa humana interagindo consigo via prompts. Agora, por outro lado, a presença desse não se faz necessária, a qual pode ser substituída por outra IA, que iterativamente realizam o processo de prompt, objetivando formar, passo a passo, um output ótimo ao contexto de sua aplicação.  

Para melhor compreensão sobre o que são Agents :
https://docs.crewai.com/core-concepts/Agents/

## Bibliotecas 📚

In [1]:
# Instalando os componentes pertinentes no notebook

!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 -q

In [2]:
# Instalando o google gemini

!pip install crewai langchain_google_genai -q

In [3]:
!pip install google-generativeai -q

In [33]:
!pip install 'crewai[tools]' -q

In [34]:
import warnings
warnings.filterwarnings('ignore')

import google.generativeai as genai

import os
import crewai
import langchain

from crewai import Agent, Task, Crew

from langchain_google_genai import ChatGoogleGenerativeAI

from crewai_tools import WebsiteSearchTool


## Criando o modelo de LLM a ser utilizado 🤖



In [35]:
os.environ["GOOGLE_API_KEY"] = ""

In [36]:
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [37]:
# Criando o modelo de llm utilizado com o google gemini.

llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-pro-latest",
    temperature = 0.8
)

### Criando os Agents

Como pode-se compreender por meio da literatura, os Agents são os recursos que permitem a automatização dos processos para eles passados, quando integrados com uma LLM. Nesse sentido, para o seu melhor desempenho, configura-se o seu papel (role), o seu objetivo (goal) e o seu contexto (backstory), na forma que cada qual, integrando-os, contextualiza melhor a LLM de uso, de modo que ela passa a prover respostas mais acuradas e adequadas ao que se espera.

In [38]:
# Planejador de conteúdo :

planner = Agent(
    role = "Planejador de conteúdo",
    goal = "Planeje o conteúdo envolvente e factualmente preciso sobre o {topico}",
    backstory = "Você está trabalhando no planejamento de um artigo de blog sobre o tópico : {topico}."
                "Você precisa coletar informações que ajudam a audiência aprender algo e tomar decisões informadas."
                "O seu trabalho é fundamental, a base, para o Escritor de Conteúdo um artigo sobre esse tópico.",
    allow_delegation = True,
    verbose = True,              # Para poder ver o Agent desempenhando a sua tarefa com base em sua regra, objetivo e backstory (contexto).
    llm = llm
)


In [39]:
# Escritor de conteúdo :

writer = Agent(
    role = "Escritor de conteúdo",
    goal = "Escreva um artigo perspicaz e factualmente acurado sobre o tópico : {topico}.",
    backstory = "Você está trabalhando na escrita de um artigo sobre a sua acurada opinião sobre o tópico : {topico}."
                "Você irá basear a sua escrita no trabalho realizado pelo Planejador de Conteúdo, que forneceu um destacado e relevante contexto sobre o tópico."
                "Você precisa seguir os objetivos principais e direções destacadas pelo Planejador de Conteúdo."
                "Você também pode fornecer objetivos e ideias parciais sobre o topico, apoiando-se no que fora trazido pelo Planejador de Conteúdo."
                "Você reconhece em seu artigo quando as suas opiniões são apenas opiniões, em oposição às declarações objetivas."
                "Escreva o artigo no idioma {idioma}.",
    allow_delegation = False,
    verbose = True,
    llm = llm
)


In [40]:
# Editor do conteúdo :

editor = Agent(
    role = "Editor",
    goal = "Edite o artigo de blog, alinhando o estilo de escrita com a organização.",
    backstory = "Você é um editor que recebeu um artigo de blog de um Escritor de Conteúdo."
                "Seu objetivo é rever o artigo de blog, buscando garantir as seguintes melhores práticas jornalísticas :"
                "Prover pontos de vistas balanceadas, quando oferecer opiniões e afirmações."
                "Evitar grandes tópicos controvérsos ou opiniões quando possível.",
    allow_delegation = False,
    verbose = True,
    llm = llm
)

## Criando as ferramentas

As ferramentas são elementos cruciais que dão aos Agents a capacidade ativa de obter informação e a gerar no meio, extendo seu âmbito de ação, para além de seu conhecimento prévio em seu treinamento.

In [41]:
# Instanciando a ferramenta que permitirá
# ao Agent realizar buscas na Internet.

# Como por padrão a ferramenta utiliza-se da OpenAi,
# tive que configurar seus parâmetros para poder utilizar
# a LLM do google.

search_tool = WebsiteSearchTool(
    config = dict(
        llm = dict(
            provider = "google",
            config = dict(
                model = "gemini-1.5-pro-latest",
                temperature = 0.8
                ),
            ),
      embedder = dict(
          provider = "google",
          config = dict(
              model = "models/embedding-001",
              task_type = "retrieval_document"
               ),
          ),
        )
    )

## Criando as tarefas

As tarefas se referem ao o que deseja-se que os Agents façam. Para a melhor qualidade de saída, recomenda-se que informe ao Agent uma descrição de sua atividade, passo a passo, de forma clara e corente, o que ele precisa realizar, informando em conjunto com o resultado esperado. Para a sua implementação, ver abaixo.

In [42]:
# Tarefa - planejar

plan = Task(
    description=(
        "1. Tenha como prioridade as últimas trends, palavras chaves e notícias importantes sobre o {topico}. \n"
        "2. Identifique a sua audiência alvo, considerando seus interesses e pontos de apelo."
        "3. Desenvolva um conteúdo detalhado, destacando os pontos principais para uma introdução e pontos chave."
        "4. Inclua SEO keywords e relevantes dados ou fontes, que podem servir de repertório à criação do artigo do blog."
    ),
    expected_output = "Um compreensível documento de planejamento de conteúdo, com pontos de destaque, audiência, SEO keywords e fontes utilizadas.",
    tools = [search_tool],
    agent = planner
)

In [43]:
# Tarefa - escrever

write = Task(
    description=(
        "1. Use o conteúdo planejado para criar um artigo de blog atrativo sobre o tópico {topico}. \n"
        "2. Incorpore SEO keywords naturalmente."
        "3. Crie Seções/Subtítulos que sejam nomeadas que sejam atrativos, buscando despertar o interesse no leitor."
        "4. Garanta que o artigo esteja estruturado com uma envolvente introdução, inpirador desenvolvimento e com uma conclusão sumarizada, que forneça uma síntese do que fora escrito."
        "5. Corrija erros gramáticos e alinhe os textos com o tom da marca."
    ),
    expected_output = "Um artigo de blog bem escrito, em formato markdown, pronto para publicação, com cada seção tendo de 2 a 3 parágrafos.",
    agent = writer
)

In [44]:
# Tarefa - editar

edit = Task(
    description=(
        "Analise o artigo de blog obtido, revisando possíveis erros gramaticais ou ortográficos e os corrija."
        "Alinhe o texto com o tom de voz da marca"),
    expected_output = "Um artigo de blog bem escrito, em formato markdown, pronto para publicação, com cada seção tendo de 2 a 3 parágrafos.",
    agent = editor
)

### Crew (Integrando os Agents com suas tarefas, produzindo um resultado final)

Essa etapa se refere à integração dos Agents com suas tarefas, em ordem de obter a saída final do que fora estipulado : um artígo atraente sobre um tópico informado.

Para isso, passa-se as tarefas em ordem cronológca, uma vez que uma influi na outra, em ordem que a saída de uma se torna a entrada de outra e assim por diante.

Para melhor compreensão ver a célula abaixo.

In [45]:
crew = Crew(
    agents = [planner, writer, editor],
    tasks = [plan, write, edit],
    verbose = 2, # O valor 2 indica quae queremos ver a saída, em debug, completa de cada etapa da sequência : ( planner - plan ) -> ( writer - write ) -> ( editor - edit )
    memory = True,
    embedder={
            "provider": "google",
            "config": {
                "model": "models/embedding-001",
                "task_type": "retrieval_document",
                "title": "Embeddings for Embedchain"
            }
    }
)

In [46]:
# Preparando os inputs para os Agents.

# Para isso, crio um dicionário, como pode ser observado
# na célula abaixo.

inputs = dict(
    topico = "Inteligência Artificial sob o ponto de vista filosófico.",
    idioma = "Português"
)

In [47]:
inputs

{'topico': 'Inteligência Artificial sob o ponto de vista filosófico.',
 'idioma': 'Português'}

In [48]:
# "Colocando os Agents para funcionar" :

result = crew.kickoff(inputs=inputs)


 [DEBUG]: == Working Agent: Planejador de conteúdo
 [INFO]: == Starting Task: 1. Tenha como prioridade as últimas trends, palavras chaves e notícias importantes sobre o Inteligência Artificial sob o ponto de vista filosófico.. 
2. Identifique a sua audiência alvo, considerando seus interesses e pontos de apelo.3. Desenvolva um conteúdo detalhado, destacando os pontos principais para uma introdução e pontos chave.4. Inclua SEO keywords e relevantes dados ou fontes, que podem servir de repertório à criação do artigo do blog.


> Entering new CrewAgentExecutor chain...
Thought: I need to gather information on the latest trends and keywords related to AI from a philosophical perspective.  A good starting point would be resources like Stanford Encyclopedia of Philosophy or similar academic resources. 
Action: Search in a specific website
Action Input: {"search_query": "artificial intelligence philosophy", "website": "https://plato.stanford.edu/"}  

Relevant Content:




Thought: I need to gather information on the latest trends and keywords related to AI from a philosophical perspective.  A good starting point would be resources like Stanford Encyclopedia of Philosophy or similar academic resources. 
Action: Search in a specific website
Action Input: {"search_query": "artificial intelligence philosophy", "website": "https://plato.stanford.edu/"}  

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to find trending topics, but I don't have a specific website for that. I should ask the Content Writer if they know any resources for this. 
Action: Ask question to co-worker
Action Input: {"coworker": "Escritor de conteúdo", "question": "Do you know of any websites or resources that track trending topics specifically related to AI and philosophy?", "context": "I need to gather information about the latest trends and keywords related to AI from a philosophical perspective. We're creating 



> Entering new CrewAgentExecutor chain...


Thought: I can definitely help my coworker find resources for trending topics in AI and philosophy. 

Final Answer:  Hey! You're looking for trending topics in AI and philosophy for our blog post, right? That's a super interesting area. While there isn't one magic website dedicated to tracking trending philosophical topics *within* AI, here's how I'd approach it:

**1. Broad Trend Trackers (then narrow down to AI & Philosophy):**

* **Google Trends:** Start broad by searching "AI" or "artificial intelligence" and see what related queries pop up. Look for philosophy-related terms in the results. You can also directly search for terms like "AI ethics," "AI consciousness," etc., and see their trend data. 
* **Exploding Topics:** This site analyzes millions of data points to identify trending topics across various categories. Search for "AI" and filter down to categories like "Tech" or "Science" and see what philosophical angles emerge. 
* **Twitter:**  Twitter is great for real-time trend

Thought: The Content Writer provided some excellent resources and strategies for identifying trending topics. I should start by exploring Google Trends and Exploding Topics.
Action: Search in a specific website
Action Input: {"search_query": "artificial intelligence", "website": "https://trends.google.com/"} 

Relevant Content:




...

Thought: I can identify several target audiences based on the provided information.  Each group has distinct interests within the broader topic of AI and philosophy, offering opportunities for tailored content.

Final Answer: 

## Potential Target Audiences for a Blog Post on AI from a Philosophical Perspective:

**1. Students and Academics in Philosophy and Technology Studies:**

* **Interests:** Deeply engaged in theoretical discussions about consciousness, ethics, and the implications of AI for humanity.
* **Appeal:**  Content exploring different philosophical frameworks (e.g., utilitarianism, existentialism) applied to AI, analysis of seminal works on AI philosophy, discussions on AI's potential impact on our understanding of concepts like intelligence, consciousness, and creativity.

**2.  Tech Professionals and AI Enthusiasts:**

* **Interests:** Curious about the ethical implications of their work, the future of AI, and the philosophical underpinnings of the technology they are 

Thought: The Content Writer provided a good list of potential target audiences. Now I need to develop a detailed content outline, including key points for the introduction and main sections. I also need to compile relevant SEO keywords and data sources.
Action: Delegate work to co-worker
Action Input: {"coworker": "Escritor de conteúdo", "task": "Develop a detailed content outline for a blog post on AI from a philosophical perspective, including key points for the introduction and main sections, relevant SEO keywords, and data sources.", "context": "Here are the potential target audiences for this blog post:\n\n**Target Audiences:**\n\n* Students and Academics in Philosophy and Technology Studies\n* Tech Professionals and AI Enthusiasts\n* General Audience Interested in the Future and Ethics of Technology\n* Artists and Creatives Concerned about AI's Impact on Creativity\n* Educators and Policy Makers\n\n**Trending Topics:**\n\n* AI Ethics\n* AI Consciousness\n* AI Image Generation\n* 

Thought: The Content Writer provided a well-structured content outline and relevant SEO keywords. I should ask the Editor to review it for clarity, accuracy, and completeness.
Action: Ask question to co-worker
Action Input: {"coworker": "Editor", "question": "Can you please review this content outline for a blog post on AI from a philosophical perspective, ensuring its clarity, accuracy, and completeness?", "context": "## Content Outline: The Philosophical Labyrinth of Artificial Intelligence

**I. Introduction**

*  **Hook:** Start with a captivating question or scenario about AI, like a self-driving car facing an ethical dilemma, or an AI-generated artwork challenging our notion of creativity.
*  **Context:** Briefly introduce AI and its growing presence in various aspects of life.
*  **Thesis Statement:**  This article delves into the philosophical labyrinth of AI, exploring questions of consciousness, ethics, and the very definition of intelligence in the face of these rapidly evol

Thought: The Content Writer provided a well-structured content outline and relevant SEO keywords. I should ask the Editor to review it for clarity, accuracy, and completeness. 
Action: Ask question to co-worker
Action Input: {"coworker": "Editor", "question": "Can you please review this content outline for a blog post on AI from a philosophical perspective, ensuring its clarity, accuracy, and completeness?", "context": "## Content Outline: The Philosophical Labyrinth of Artificial Intelligence

**I. Introduction**

*  **Hook:** Start with a captivating question or scenario about AI, like a self-driving car facing an ethical dilemma, or an AI-generated artwork challenging our notion of creativity.
*  **Context:** Briefly introduce AI and its growing presence in various aspects of life.
*  **Thesis Statement:**  This article delves into the philosophical labyrinth of AI, exploring questions of consciousness, ethics, and the very definition of intelligence in the face of these rapidly evo

Thought: The Content Writer provided a well-structured content outline and relevant SEO keywords. I should ask the Editor to review it for clarity, accuracy, and completeness. 
Action: Ask question to co-worker
Action Input: {"coworker": "Editor", "question": "Can you please review this content outline for a blog post on AI from a philosophical perspective, ensuring its clarity, accuracy, and completeness?", "context": "Here is the content outline: ## Content Outline: The Philosophical Labyrinth of Artificial Intelligence\n\n**I. Introduction**\n\n*  **Hook:** Start with a captivating question or scenario about AI, like a self-driving car facing an ethical dilemma, or an AI-generated artwork challenging our notion of creativity.\n*  **Context:** Briefly introduce AI and its growing presence in various aspects of life.\n*  **Thesis Statement:**  This article delves into the philosophical labyrinth of AI, exploring questions of consciousness, ethics, and the very definition of intelligen

In [49]:
from IPython.display import Markdown

# Formatando a saída em Markdown, para que seja mais atraente visualmente.

Markdown(result)

## Mergulhando no Labirinto Filosófico da Inteligência Artificial

**Imagine um juiz de IA presidindo um tribunal, sua voz sintetizada pronunciando um veredito. Seria este o futuro da justiça, ou um cenário distópico onde máquinas decidem nossos destinos?** A inteligência artificial, antes restrita à ficção científica, está rapidamente se entrelaçando com a realidade. De carros autônomos a diagnósticos médicos, a influência da IA é inegável. Este artigo mergulha nas profundezas filosóficas da IA, confrontando questões de consciência, dilemas éticos e a própria essência da inteligência na era das máquinas pensantes. Se você é um entusiasta de tecnologia ponderando sobre o futuro, um filósofo lutando com questões existenciais ou simplesmente sente curiosidade pela linha que divide o humano da máquina, esta exploração é para você.

## Podem as Máquinas Pensar? Desvendando o Enigma da Consciência na IA

A busca por criar máquinas pensantes não é nova. Filósofos como Alan Turing, com seu teste inovador para inteligência de máquina, plantaram as primeiras sementes dessa busca. Mas a pergunta central persiste: podem as máquinas realmente *pensar*?

O **computacionalismo** argumenta que a mente humana opera como um programa de computador complexo. Se assim for, poderia a IA sofisticada um dia alcançar a consciência? Por outro lado, as **teorias da incorporação** defendem que a consciência emerge da interação complexa entre nossos corpos físicos, o ambiente e nossas experiências. A IA, desprovida dessa incorporação física, pode realmente ser consciente?

E mesmo que a IA imite perfeitamente o comportamento humano, podemos realmente afirmar que ela experimenta sentimentos subjetivos, emoções ou "qualia"? Este é o **problema difícil da consciência**, um mistério que ainda desafia a compreensão humana. Ao analisarmos os sistemas de IA contemporâneos, questionamos se eles exibem algum indício de consciência e apresentamos diferentes perspectivas de especialistas sobre este debate acalorado.

## A Encruzilhada Ética: Navegando na Paisagem Moral da IA

A IA, embora aparentemente objetiva, pode herdar e amplificar os **vieses** presentes nos dados que a treinam. Isso pode levar a resultados injustos ou discriminatórios, particularmente em áreas como justiça criminal, processos de contratação e aprovação de empréstimos. 

Outro dilema ético surge com o desenvolvimento de **sistemas de armas letais autônomas (LAWs)**. Podemos confiar às máquinas o poder de matar? O que acontece quando a IA falha em campo de batalha?

Além disso, a **automação impulsionada pela IA** está pronta para transformar o mercado de trabalho, com potencial para deslocar milhões de trabalhadores. Como podemos garantir uma transição justa e lidar com as preocupações econômicas que essa mudança pode gerar?

À medida que os sistemas de IA se tornam mais sofisticados e autônomos, atribuir **responsabilidade** por suas ações se torna cada vez mais complexo. Quem é o responsável quando uma IA toma uma decisão que muda vidas: o programador, o usuário ou a própria máquina? A necessidade de diretrizes éticas, regulamentações e cooperação internacional para guiar o desenvolvimento da IA em uma direção responsável e benéfica é mais urgente do que nunca.

## A IA e a Fonte da Criatividade: Disruptora ou Musa?

Prepare-se para se surpreender com exemplos de arte, música e literatura geradas por IA, expandindo os limites da criação por máquinas. Mas será a IA capaz de alcançar a **expressão artística genuína**, ou a criatividade é um traço exclusivamente humano?

Em um mundo onde a IA pode gerar conteúdo aparentemente original, como redefinimos a **autoria**? Quem detém os direitos autorais de obras geradas por IA? A IA tornará os artistas humanos obsoletos, aprimorará suas habilidades ou dará origem a formas de arte completamente novas?  

## Olhando para o Horizonte: Reflexões e Próximos Passos

A exploração da IA está apenas começando. As questões filosóficas que ela levanta sobre inteligência, consciência, ética e a própria natureza da realidade continuarão a nos desafiar e intrigar. O futuro da IA não está predeterminado. Cabe a nós, como sociedade, moldar esse futuro por meio do pensamento crítico, do diálogo aberto e de uma abordagem ética e consciente para o desenvolvimento e a implementação dessa tecnologia poderosa.

Convidamos você a se juntar à conversa. Explore os recursos e links relacionados para aprofundar seu conhecimento e compartilhar suas próprias reflexões sobre o impacto da IA em nossas vidas e no futuro da humanidade.